In [9]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [10]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


 Разделим наши данные на тренировочную и тестовую выборки

In [11]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:

    gender, cholesterol применим OHE-кодирование
    age, height, weight, ap_hi, ap_lo - standardScaler
    gluc, smoke, alco, active - оставим пока как есть



In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion


In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

### LogisticRegression

In [32]:
%%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111
CPU times: user 30.3 s, sys: 19.2 s, total: 49.4 s
Wall time: 14.3 s


Посчитаем precision/recall/f_score       pr/rec/auc/f_score


In [7]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

In [35]:
b=1
precision_lr, recall_lr, thresholds = precision_recall_curve(y_test.values, y_score)
fscore_lr = (1+b**2)*(precision_lr * recall_lr) / (b**2*precision_lr + recall_lr)
# locate the index of the largest f score
ix = np.argmax(fscore_lr)
max_fscore_lr = fscore_lr[ix]
max_precision_lr = precision_lr[ix]
max_recall_lr =  recall_lr[ix]
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore_lr[ix],
                                                                        precision_lr[ix],
                                                                        recall_lr[ix]))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838


In [13]:
from sklearn.metrics import roc_auc_score, log_loss


roc_auc_lr = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
print("roc auc score: {}".format(roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])))
print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=classifier.predict_proba(X_test)[:,1])))

roc auc score: 0.7840347790421852
log loss score: 0.5779604008230668


In [ ]:
pr/rec/auc/f_score

### RandomForest

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10.5 µs
CV score is 0.7734501681056019+-0.007171140345435727


In [41]:
b=1
precision_rf, recall_rf, thresholds = precision_recall_curve(y_test.values, y_score)
fscore_rf = (1+b**2)*(precision_rf * recall_rf) / (b**2*precision_rf + recall_rf)
# locate the index of the largest f score
ix = np.argmax(fscore_rf)
max_fscore_rf = fscore_rf[ix]
max_precision_rf = precision_rf[ix]
max_recall_rf =  recall_rf[ix]
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore_rf[ix],
                                                                        precision_rf[ix],
                                                                        recall_rf[ix]))

Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816


In [42]:
roc_auc_rf = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
print("roc auc score: {}".format(roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])))
print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=classifier.predict_proba(X_test)[:,1])))

roc auc score: 0.7710366181802983
log loss score: 0.5992984853728378


### CatBoost

In [21]:
from catboost import CatBoostClassifier

In [22]:
%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs
Learning rate set to 0.054384
0:	learn: 0.6773390	total: 136ms	remaining: 2m 15s
1:	learn: 0.6632637	total: 159ms	remaining: 1m 19s
2:	learn: 0.6504265	total: 186ms	remaining: 1m 1s
3:	learn: 0.6396772	total: 227ms	remaining: 56.5s
4:	learn: 0.6298071	total: 256ms	remaining: 51s
5:	learn: 0.6212486	total: 284ms	remaining: 47.1s
6:	learn: 0.6132777	total: 314ms	remaining: 44.6s
7:	learn: 0.6061517	total: 338ms	remaining: 41.9s
8:	learn: 0.6004188	total: 360ms	remaining: 39.6s
9:	learn: 0.5947811	total: 380ms	remaining: 37.6s
10:	learn: 0.5895252	total: 405ms	remaining: 36.4s
11:	learn: 0.5850214	total: 426ms	remaining: 35.1s
12:	learn: 0.5811643	total: 462ms	remaining: 35s
13:	learn: 0.5777466	total: 492ms	remaining: 34.6s
14:	learn: 0.5744545	total: 519ms	remaining: 34.1s
15:	learn: 0.5716082	total: 538ms	remaining: 33.1s
16:	learn: 0.5690561	total: 560ms	remaining: 32.4s
17:	learn: 0.5667366	total: 590ms	remaining: 32.2s
18

163:	learn: 0.5318894	total: 5.09s	remaining: 25.9s
164:	learn: 0.5318391	total: 5.15s	remaining: 26.1s
165:	learn: 0.5317765	total: 5.18s	remaining: 26s
166:	learn: 0.5317057	total: 5.2s	remaining: 26s
167:	learn: 0.5316778	total: 5.23s	remaining: 25.9s
168:	learn: 0.5316436	total: 5.26s	remaining: 25.9s
169:	learn: 0.5316098	total: 5.29s	remaining: 25.8s
170:	learn: 0.5315836	total: 5.34s	remaining: 25.9s
171:	learn: 0.5315242	total: 5.39s	remaining: 25.9s
172:	learn: 0.5315059	total: 5.43s	remaining: 25.9s
173:	learn: 0.5314714	total: 5.45s	remaining: 25.9s
174:	learn: 0.5314260	total: 5.48s	remaining: 25.8s
175:	learn: 0.5313816	total: 5.5s	remaining: 25.8s
176:	learn: 0.5313466	total: 5.53s	remaining: 25.7s
177:	learn: 0.5313263	total: 5.55s	remaining: 25.6s
178:	learn: 0.5312659	total: 5.59s	remaining: 25.6s
179:	learn: 0.5312238	total: 5.61s	remaining: 25.6s
180:	learn: 0.5311869	total: 5.65s	remaining: 25.6s
181:	learn: 0.5311361	total: 5.72s	remaining: 25.7s
182:	learn: 0.5310

326:	learn: 0.5247204	total: 10s	remaining: 20.6s
327:	learn: 0.5247054	total: 10.1s	remaining: 20.8s
328:	learn: 0.5246505	total: 10.2s	remaining: 20.8s
329:	learn: 0.5246071	total: 10.2s	remaining: 20.7s
330:	learn: 0.5245672	total: 10.2s	remaining: 20.7s
331:	learn: 0.5245204	total: 10.2s	remaining: 20.6s
332:	learn: 0.5244436	total: 10.3s	remaining: 20.6s
333:	learn: 0.5244167	total: 10.3s	remaining: 20.5s
334:	learn: 0.5244102	total: 10.3s	remaining: 20.5s
335:	learn: 0.5243442	total: 10.3s	remaining: 20.4s
336:	learn: 0.5243155	total: 10.4s	remaining: 20.4s
337:	learn: 0.5242896	total: 10.4s	remaining: 20.3s
338:	learn: 0.5242596	total: 10.4s	remaining: 20.3s
339:	learn: 0.5242142	total: 10.4s	remaining: 20.3s
340:	learn: 0.5241928	total: 10.5s	remaining: 20.2s
341:	learn: 0.5241201	total: 10.5s	remaining: 20.2s
342:	learn: 0.5240784	total: 10.6s	remaining: 20.2s
343:	learn: 0.5240313	total: 10.6s	remaining: 20.2s
344:	learn: 0.5239662	total: 10.7s	remaining: 20.3s
345:	learn: 0.

487:	learn: 0.5185118	total: 16.1s	remaining: 16.9s
488:	learn: 0.5184665	total: 16.3s	remaining: 17s
489:	learn: 0.5184205	total: 16.3s	remaining: 17s
490:	learn: 0.5183934	total: 16.4s	remaining: 17s
491:	learn: 0.5183519	total: 16.5s	remaining: 17s
492:	learn: 0.5183194	total: 16.5s	remaining: 17s
493:	learn: 0.5182658	total: 16.6s	remaining: 17s
494:	learn: 0.5182277	total: 16.6s	remaining: 17s
495:	learn: 0.5181799	total: 16.7s	remaining: 16.9s
496:	learn: 0.5181359	total: 16.7s	remaining: 16.9s
497:	learn: 0.5181216	total: 16.8s	remaining: 16.9s
498:	learn: 0.5181072	total: 16.8s	remaining: 16.9s
499:	learn: 0.5180865	total: 16.9s	remaining: 16.9s
500:	learn: 0.5180441	total: 16.9s	remaining: 16.9s
501:	learn: 0.5179973	total: 17s	remaining: 16.8s
502:	learn: 0.5179674	total: 17.1s	remaining: 16.8s
503:	learn: 0.5179524	total: 17.1s	remaining: 16.8s
504:	learn: 0.5179222	total: 17.2s	remaining: 16.8s
505:	learn: 0.5179023	total: 17.2s	remaining: 16.8s
506:	learn: 0.5178754	total:

648:	learn: 0.5130780	total: 28s	remaining: 15.2s
649:	learn: 0.5130363	total: 28.2s	remaining: 15.2s
650:	learn: 0.5130097	total: 28.3s	remaining: 15.2s
651:	learn: 0.5129806	total: 28.4s	remaining: 15.2s
652:	learn: 0.5129622	total: 28.6s	remaining: 15.2s
653:	learn: 0.5129294	total: 28.7s	remaining: 15.2s
654:	learn: 0.5128828	total: 28.8s	remaining: 15.2s
655:	learn: 0.5128367	total: 28.9s	remaining: 15.2s
656:	learn: 0.5128303	total: 29s	remaining: 15.1s
657:	learn: 0.5128135	total: 29.1s	remaining: 15.1s
658:	learn: 0.5127800	total: 29.2s	remaining: 15.1s
659:	learn: 0.5127763	total: 29.4s	remaining: 15.2s
660:	learn: 0.5127324	total: 29.5s	remaining: 15.1s
661:	learn: 0.5126803	total: 29.7s	remaining: 15.2s
662:	learn: 0.5126673	total: 29.9s	remaining: 15.2s
663:	learn: 0.5126231	total: 29.9s	remaining: 15.1s
664:	learn: 0.5125898	total: 30s	remaining: 15.1s
665:	learn: 0.5125224	total: 30.1s	remaining: 15.1s
666:	learn: 0.5124903	total: 30.2s	remaining: 15.1s
667:	learn: 0.5124

...

249:	learn: 0.5277457	total: 6.37s	remaining: 19.1s
250:	learn: 0.5277085	total: 6.39s	remaining: 19.1s
251:	learn: 0.5276690	total: 6.42s	remaining: 19s
252:	learn: 0.5276235	total: 6.44s	remaining: 19s
253:	learn: 0.5275462	total: 6.46s	remaining: 19s
254:	learn: 0.5275101	total: 6.49s	remaining: 18.9s
255:	learn: 0.5274850	total: 6.51s	remaining: 18.9s
256:	learn: 0.5274631	total: 6.53s	remaining: 18.9s
257:	learn: 0.5274071	total: 6.55s	remaining: 18.8s
258:	learn: 0.5274001	total: 6.57s	remaining: 18.8s
259:	learn: 0.5273670	total: 6.59s	remaining: 18.8s
260:	learn: 0.5273365	total: 6.62s	remaining: 18.7s
261:	learn: 0.5272673	total: 6.64s	remaining: 18.7s
262:	learn: 0.5272186	total: 6.66s	remaining: 18.7s
263:	learn: 0.5271846	total: 6.68s	remaining: 18.6s
264:	learn: 0.5271476	total: 6.71s	remaining: 18.6s
265:	learn: 0.5271300	total: 6.73s	remaining: 18.6s
266:	learn: 0.5270722	total: 6.75s	remaining: 18.5s
267:	learn: 0.5270424	total: 6.78s	remaining: 18.5s
268:	learn: 0.5269

413:	learn: 0.5212371	total: 10.9s	remaining: 15.4s
414:	learn: 0.5212124	total: 10.9s	remaining: 15.4s
415:	learn: 0.5211706	total: 10.9s	remaining: 15.4s
416:	learn: 0.5211399	total: 11s	remaining: 15.3s
417:	learn: 0.5210827	total: 11s	remaining: 15.3s
418:	learn: 0.5210717	total: 11s	remaining: 15.3s
419:	learn: 0.5210444	total: 11s	remaining: 15.3s
420:	learn: 0.5210415	total: 11.1s	remaining: 15.2s
421:	learn: 0.5209978	total: 11.1s	remaining: 15.2s
422:	learn: 0.5209639	total: 11.1s	remaining: 15.2s
423:	learn: 0.5209100	total: 11.2s	remaining: 15.1s
424:	learn: 0.5208813	total: 11.2s	remaining: 15.1s
425:	learn: 0.5208523	total: 11.2s	remaining: 15.1s
426:	learn: 0.5208314	total: 11.2s	remaining: 15.1s
427:	learn: 0.5207813	total: 11.2s	remaining: 15s
428:	learn: 0.5207201	total: 11.3s	remaining: 15s
429:	learn: 0.5206404	total: 11.3s	remaining: 15s
430:	learn: 0.5206125	total: 11.3s	remaining: 14.9s
431:	learn: 0.5205769	total: 11.3s	remaining: 14.9s
432:	learn: 0.5205286	tota

576:	learn: 0.5158878	total: 15.5s	remaining: 11.4s
577:	learn: 0.5158585	total: 15.5s	remaining: 11.3s
578:	learn: 0.5158319	total: 15.6s	remaining: 11.3s
579:	learn: 0.5157904	total: 15.6s	remaining: 11.3s
580:	learn: 0.5157519	total: 15.7s	remaining: 11.3s
581:	learn: 0.5157064	total: 15.7s	remaining: 11.3s
582:	learn: 0.5156476	total: 15.7s	remaining: 11.2s
583:	learn: 0.5156241	total: 15.8s	remaining: 11.2s
584:	learn: 0.5155837	total: 15.8s	remaining: 11.2s
585:	learn: 0.5155538	total: 15.8s	remaining: 11.2s
586:	learn: 0.5155050	total: 15.8s	remaining: 11.2s
587:	learn: 0.5154630	total: 15.9s	remaining: 11.1s
588:	learn: 0.5154607	total: 15.9s	remaining: 11.1s
589:	learn: 0.5154269	total: 15.9s	remaining: 11.1s
590:	learn: 0.5153924	total: 16s	remaining: 11.1s
591:	learn: 0.5153699	total: 16s	remaining: 11s
592:	learn: 0.5153301	total: 16s	remaining: 11s
593:	learn: 0.5153093	total: 16.1s	remaining: 11s
594:	learn: 0.5152964	total: 16.1s	remaining: 10.9s
595:	learn: 0.5152629	to

741:	learn: 0.5105834	total: 20s	remaining: 6.96s
742:	learn: 0.5105480	total: 20s	remaining: 6.93s
743:	learn: 0.5104936	total: 20.1s	remaining: 6.91s
744:	learn: 0.5104421	total: 20.1s	remaining: 6.88s
745:	learn: 0.5104187	total: 20.1s	remaining: 6.85s
746:	learn: 0.5103823	total: 20.1s	remaining: 6.82s
747:	learn: 0.5103598	total: 20.2s	remaining: 6.79s
748:	learn: 0.5103154	total: 20.2s	remaining: 6.76s
749:	learn: 0.5102804	total: 20.2s	remaining: 6.74s
750:	learn: 0.5102451	total: 20.2s	remaining: 6.71s
751:	learn: 0.5102118	total: 20.3s	remaining: 6.68s
752:	learn: 0.5102105	total: 20.3s	remaining: 6.66s
753:	learn: 0.5101847	total: 20.3s	remaining: 6.63s
754:	learn: 0.5101649	total: 20.3s	remaining: 6.6s
755:	learn: 0.5101433	total: 20.4s	remaining: 6.57s
756:	learn: 0.5101290	total: 20.4s	remaining: 6.54s
757:	learn: 0.5100919	total: 20.4s	remaining: 6.52s
758:	learn: 0.5100829	total: 20.4s	remaining: 6.49s
759:	learn: 0.5100609	total: 20.5s	remaining: 6.46s
760:	learn: 0.510

902:	learn: 0.5058376	total: 24.3s	remaining: 2.61s
903:	learn: 0.5057922	total: 24.4s	remaining: 2.59s
904:	learn: 0.5057692	total: 24.4s	remaining: 2.56s
905:	learn: 0.5057607	total: 24.4s	remaining: 2.53s
906:	learn: 0.5057440	total: 24.4s	remaining: 2.51s
907:	learn: 0.5057141	total: 24.5s	remaining: 2.48s
908:	learn: 0.5057134	total: 24.5s	remaining: 2.45s
909:	learn: 0.5057010	total: 24.5s	remaining: 2.42s
910:	learn: 0.5056853	total: 24.5s	remaining: 2.4s
911:	learn: 0.5056809	total: 24.6s	remaining: 2.37s
912:	learn: 0.5056596	total: 24.6s	remaining: 2.34s
913:	learn: 0.5056127	total: 24.6s	remaining: 2.32s
914:	learn: 0.5055930	total: 24.6s	remaining: 2.29s
915:	learn: 0.5055458	total: 24.7s	remaining: 2.26s
916:	learn: 0.5055086	total: 24.7s	remaining: 2.23s
917:	learn: 0.5054829	total: 24.7s	remaining: 2.21s
918:	learn: 0.5054822	total: 24.8s	remaining: 2.18s
919:	learn: 0.5054518	total: 24.8s	remaining: 2.15s
920:	learn: 0.5054331	total: 24.8s	remaining: 2.13s
921:	learn: 0

In [29]:
b=1
precision_cb, recall_cb, thresholds = precision_recall_curve(y_test.values, y_score)
fscore_cb = (1+b**2)*(precision_cb * recall_cb) / (b**2*precision_cb + recall_cb)
# locate the index of the largest f score
ix = np.argmax(fscore_cb)
max_fscore_cb = fscore_cb[ix]
max_precision_cb = precision_cb[ix]
max_recall_cb =  recall_cb[ix]
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore_cb[ix],
                                                                        precision_cb[ix],
                                                                        recall_cb[ix]))

Best Threshold=0.383138, F-Score=0.740, Precision=0.690, Recall=0.797


In [24]:
roc_auc_cb = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
print("roc auc score: {}".format(roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])))
print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=classifier.predict_proba(X_test)[:,1])))

roc auc score: 0.8010916943582349
log loss score: 0.5416700716875228


## 3. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [48]:
scores = pd.DataFrame({'Type': ['LinearRegression', 'RandomForest', 'CatBoostClassifier'],
                       'roc auc':[roc_auc_lr,roc_auc_rf, roc_auc_cb],
                       'precison': [max_precision_lr, max_precision_rf, max_precision_cb], 
                       'recall':[max_recall_lr, max_recall_rf, max_recall_cb], 
                       'f_score':[max_fscore_lr, max_fscore_rf, max_fscore_cb]})

In [49]:
scores.head()


,Type,roc auc,precison,recall,f_score
0,LinearRegression,0.784035,0.647431,0.837558,0.730323
1,RandomForest,0.771037,0.642669,0.815553,0.718863
2,CatBoostClassifier,0.801092,0.689686,0.797350,0.739621


## 4. сделать выводы о том, какая модель справилась с задачей лучше других

#### Лучшей оказалась модель CatBoostClassifier.